In [57]:
# import graphviz
# from abc import ABC, abstractmethod
# from typing import List, Tuple


# class Node(ABC):
#     def __init__(self, name, style="filled"):
#         self.name = name
#         self.style = style
#         self.children = []

#     def add_child(self, child):
#         self.children.append(child)

#     @property
#     @abstractmethod
#     def color(self):
#         pass


# class InputDataNode(Node):
#     @property
#     def color(self):
#         return "lightgreen"


# class OperationNode(Node):
#     @property
#     def color(self):
#         return "skyblue"


# class IntermediateDataNode(Node):
#     @property
#     def color(self):
#         return "orange"


# class ResultDataNode(Node):
#     @property
#     def color(self):
#         return "deeppink"


# class ComputationalGraph:
#     def __init__(self, nodes: List[Node] =[], edges: List[Tuple[str, str]] = [],  filename='output/filled_colorful_organogram.gv'):
#         self.nodes: List[Node] = nodes
#         self.edges: List[Tuple[str, str]] = edges
#         self.graph = graphviz.Digraph(filename=filename)

#         for parent, child in edges:
#             parent_node = next((n for n in self.nodes if n.name == parent), None)
#             child_node = next((n for n in self.nodes if n.name == child), None)
#             if parent_node and child_node:
#                 parent_node.add_child(child_node)
#             else:
#                 print(
#                     f"Error: Can't find node by name '{parent}' or '{child}'")

#     def add_node(self, node):
#         self.nodes.append(node)

#     def generate_graph(self):
#         for node in self.nodes:
#             self.graph.node(node.name, style=node.style, fillcolor=node.color, color='white')
#             for child in node.children:
#                 self.graph.edge(node.name, child.name)
#         return self.graph


# nodes = [
#     InputDataNode("mobility_api_url"),
#     InputDataNode("france_shp_uri"),
#     OperationNode("load_mobility_data"),
#     OperationNode("load_france_shp"),
#     IntermediateDataNode("mobility_df"),
#     IntermediateDataNode("france_gdf"),
#     OperationNode("filter_mobility_data"),
#     OperationNode("merge_change_rates_with_gdf"),
#     IntermediateDataNode("monthly_change_rates_df"),
#     IntermediateDataNode("merged_gdf"),
#     OperationNode("compute_monthly_change_rates"),
#     ResultDataNode("france_mobility_df"),
#     OperationNode("visualize_monthly_change_rates"),
#     ResultDataNode("france_map_matrix"),
#     OperationNode("draw_line_chart"),
#     ResultDataNode("line_chart")
# ]

# edges = [
#     ("mobility_api_url", "load_mobility_data"),
#     ("load_mobility_data", "mobility_df"),
#     ("france_shp_uri", "load_france_shp"),
#     ("load_france_shp", "france_gdf"),
#     ("mobility_df", "filter_mobility_data"),
#     ("filter_mobility_data", "france_mobility_df"),
#     ("france_gdf", "merge_change_rates_with_gdf"),
#     ("france_mobility_df", "compute_monthly_change_rates"),
#     ("compute_monthly_change_rates", "monthly_change_rates_df"),
#     ("monthly_change_rates_df", "merge_change_rates_with_gdf"),
#     ("merge_change_rates_with_gdf", "merged_gdf"),
#     ("merged_gdf", "visualize_monthly_change_rates"),
#     ("visualize_monthly_change_rates", "france_map_matrix"),
#     ("monthly_change_rates_df", "draw_line_chart"),
#     ("draw_line_chart", "line_chart")
# ]

# computational_graph = ComputationalGraph(nodes, edges)

# graph = computational_graph.generate_graph()
# graph

In [58]:
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from enum import Enum
from typing import Any, Type, Union

class InputDataNode(BaseModel):
    pass

class OperationNode(BaseModel):
    pass

class IntermediateDataNode(BaseModel):
    pass

class ResultDataNode(BaseModel):
    pass

class NodeType(str, Enum):
    input_data = 'input_data'
    intermediary_data = 'intermediary_data'
    operation = 'operation'
    result_data = 'result_data'


class Node(BaseModel):
    name: str
    node_type: NodeType

class Edge(BaseModel):
    tail_name: str
    head_name: str

In [59]:
import graphviz
from typing import List, Dict

class ComputationalGraph:
    color_map: Dict[NodeType, str] = {
        'input_data': 'lightgreen',
        'intermediary_data': 'orange',
        'operation': 'skyblue',
        'result_data': 'deeppink'
    }

    def __init__(self, nodes: List[Node] = [], edges: List[Edge] = [],  filename='output/filled_colorful_organogram.gv'):
        self.nodes: List[Node] = nodes
        self.edges: List[Edge] = edges
        self.graph = graphviz.Digraph(filename=filename)

    def add_node(self, node: Node):
        self.nodes.append(node)

    def generate_graph(self):
        for node in self.nodes:
            self.graph.node(node.name, style='filled', fillcolor=self.color_map[node.node_type])
        for edge in self.edges:
            self.graph.edge(edge.tail_name, edge.head_name)
        return self.graph

In [60]:
class CreateComputationalGraphInput(BaseModel):
    nodes: List[Node] = Field(..., description='List of nodes')
    edges: List[Edge] = Field(..., description='List of edges between nodes')


class CreateComputationalGraphTool(BaseTool):
    name: str = 'create_computational_graph'
    args_schema: Type[BaseModel] = CreateComputationalGraphInput
    description: str = 'Use this to create computational graph meant to help solve GIS-related problems.'

    def _run(self, nodes: List[Node], edges: List[Edge], *args: Any, **kwargs: Any) -> Any:
        computational_graph = ComputationalGraph(nodes, edges)
        computational_graph.generate_graph().view()
        return (
            f'nodes: {nodes}\n'
            f'edges: {edges}'
        )

In [62]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
from langchain.agents import create_openai_tools_agent, AgentExecutor

load_dotenv('../../../.env')

system_message = """You to create a computation graph for a GIS-related question.
You have four types of nodes () that all implement the abstract Node class. 

class Node(ABC):
    def __init__(self, name, position, style="filled"):
        ...

Create an array of nodes and edges that looks something like this: 

nodes = [
    InputDataNode("mobility_data"),
    InputDataNode("france_data"),
    OperationNode("load_mobility_data"),
    OperationNode("load_france_data"),
    IntermediateDataNode("mobility_df"),
    IntermediateDataNode("france_gdf"),
    OperationNode("filter_mobility_data"),
    OperationNode("merge_change_rates_with_gdf"),
    IntermediateDataNode("monthly_change_rates_df"),
    IntermediateDataNode("merged_gdf"),
    OperationNode("compute_monthly_change_rates"),
    ResultDataNode("france_mobility_df"),
    OperationNode("visualize_monthly_change_rates"),
    ResultDataNode("france_map_matrix"),
    OperationNode("draw_line_chart"),
    ResultDataNode("line_chart")
]

edges = [
    Edge("mobility_data", "load_mobility_data"),
    Edge("load_mobility_data", "mobility_df"),
    Edge("france_data", "load_france_data"),
    Edge("load_france_data", "france_gdf"),
    Edge("mobility_df", "filter_mobility_data"),
    Edge("filter_mobility_data", "france_mobility_df"),
    Edge("france_gdf", "merge_change_rates_with_gdf"),
    Edge("france_mobility_df", "compute_monthly_change_rates"),
    Edge("compute_monthly_change_rates", "monthly_change_rates_df"),
    Edge("monthly_change_rates_df", "merge_change_rates_with_gdf"),
    Edge("merge_change_rates_with_gdf", "merged_gdf"),
    Edge("merged_gdf", "visualize_monthly_change_rates"),
    Edge("visualize_monthly_change_rates", "france_map_matrix"),
    Edge("monthly_change_rates_df", "draw_line_chart"),
    Edge("draw_line_chart", "line_chart")
]

The graph should solve the user's question.
"""

system_message2 = "Create a computational graph for a GIS-related question."

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_message2),
        HumanMessagePromptTemplate(prompt=PromptTemplate(
            input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

tools = [CreateComputationalGraphTool()]

agent = create_openai_tools_agent(llm=llm, prompt=prompt, tools=tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

res = agent_executor.invoke(
    {'input': 'I want to find houses within 50 meters of all roads.'})



> Entering new AgentExecutor chain...

Invoking: `create_computational_graph` with `{'nodes': [{'name': 'Roads', 'node_type': 'input_data'}, {'name': 'Houses', 'node_type': 'input_data'}, {'name': 'Buffer', 'node_type': 'operation'}, {'name': 'Intersect', 'node_type': 'operation'}, {'name': 'Result', 'node_type': 'result_data'}], 'edges': [{'tail_name': 'Roads', 'head_name': 'Buffer'}, {'tail_name': 'Houses', 'head_name': 'Intersect'}, {'tail_name': 'Buffer', 'head_name': 'Intersect'}, {'tail_name': 'Intersect', 'head_name': 'Result'}]}`


nodes: [Node(name='Roads', node_type=<NodeType.input_data: 'input_data'>), Node(name='Houses', node_type=<NodeType.input_data: 'input_data'>), Node(name='Buffer', node_type=<NodeType.operation: 'operation'>), Node(name='Intersect', node_type=<NodeType.operation: 'operation'>), Node(name='Result', node_type=<NodeType.result_data: 'result_data'>)]
edges: [Edge(tail_name='Roads', head_name='Buffer'), Edge(tail_name='Houses', head_name='Intersect'), Ed